## Final Project Submission

Please fill out:
* Student name: Solphine Joseph
* Student pace:Hybrid
* Scheduled project review date/time: 
* Instructor name: Nikita/Diana
* Blog post URL:


                     
## BUSINESS UNDERSTANDING 
INTRODUCTION

Microsoft company is a leading company in technology that develops a wide range of software,services and devices.However the company has not yet ventured into movie production .This project aims at coming up with analyis of various factors that will provide the stakeholders of the company with real_time based analysis of what to consider before launching into movie production space.Some of the factors to be analyzed include:

- Studios that are doing well in the movie industry

- The top genres that are leading in the leading movie  studios

- Analyze the relationship between the total gross  and the leadings studios 

- Perform an analysis that identifies the top performing directors 



OBJECTIVES 

- To help microsoft leverage the data analysis carried out in order to understand the preferences of their target audience .

- To identify the best directors in the space 

- To ensure the company is making profit at the end of the day


           

####

## DATA UNDERSTANDING 
Data required in this project is sourced from :

1 https://www.imdb.com/
 
2 https://www.boxofficemojo.com/date/?ref_=bo_nb_shs_tab

3 https://www.rottentomatoes.com/

4 https://www.themoviedb.org/
          
5 https://www.the-numbers.com/'




The data has been compiled from the internet and compressed to a folder that gives access to all required information.This data was preffered since it is up to date.

## METHODOLOGY

The methodology seeks to show the data cleaning and analysis procedures in this project.


In [50]:
#Import relevant libraries and connect to SQlite Database
import pandas as pd
import csv
import sqlite3
import matplotlib.pyplot as plt



1. READING THE DATA

In [51]:
#Read  movie_gross data and put it in a dataframe defined gross_data
gross_data=pd.read_csv('zippedData/bom.movie_gross.csv.gz')


In [52]:
#Read and extract imdb dzip file  into destination directory
import zipfile
# Specify the path to the zip file
zip_file_path = 'zippedData\im.db.zip'
# Open the zip file for reading
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#Extract the files to a directory
  zip_ref.extractall('destination_directory') 


In [53]:
#connect to the Database in destination_directory

# Specify the path to the SQLite database file
db_file_path = 'destination_directory/im.db'

# Connect destination_directoryto the SQLite database
connection = sqlite3.connect(db_file_path)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Now you can execute SQL queries using the cursor
# For example, you can fetch data from a table
##cursor.execute('SELECT * FROM your_table_name')

# Fetch the results of the query
#rows = cursor.fetchall()

# Print the results
#for row in rows:
   # print(row)

# Don't forget to close the cursor and the connection when you're done
#cursor.close()
#connection.close()


     1.1 Successful studios in relation to the gross proceeds

This process entails cleaning the movie gross data that is crucial in understanding which companies are doing well in relation to the their gross  income and conduct some analysis as well


        1.1.1 Cleaning the Data

In [54]:
# Display data from BOM that shows the bom.movie_gross proceeds 
gross_data

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [55]:
# Clean the data :Replace the null values in foreign gross and domestic_grosss columns
# Check 'foreign_gross'column  Before Modification
before_foreign_nulls = gross_data['foreign_gross'].isnull().sum()
before_domestic_nulls = gross_data['domestic_gross'].isnull().sum()

#Replace the null values
gross_data['foreign_gross'].fillna(0, inplace=True)
gross_data['domestic_gross'].fillna(0, inplace=True)

# Check 'foreign_gross'column  After Modification
after_foreign_nulls = gross_data['foreign_gross'].isnull().sum()
after_domestic_nulls = gross_data['domestic_gross'].isnull().sum()



In [56]:
#show the datatypes in the different columns
print(gross_data.dtypes)

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object


In [57]:
#Change the foreign_gross datatype to float 
# Check the current data type of 'foreign_gross' column before modification
before_dtype = gross_data['foreign_gross'].dtype

# Run the Code
gross_data.replace(',','', regex=True, inplace=True)
gross_data['foreign_gross']=gross_data['foreign_gross'].astype(float)

#check the data type of 'foreign_gross' column after modification
after_dtype = gross_data['foreign_gross'].dtype
after_dtype

#assert the changes made
assert after_dtype == float, "The data type of 'foreign_gross' column was not changed to float."



1.1.2 Exploratory Data Analysis of BOM Data



In [58]:
#To resolve the type error that comes about when adding the domestic and foreign gross values
# Convert any non-numeric values to NaN in 'domestic_gross' and 'foreign_gross' columns
gross_data['domestic_gross'] = pd.to_numeric(gross_data['domestic_gross'], errors='coerce')
gross_data['foreign_gross'] = pd.to_numeric(gross_data['foreign_gross'], errors='coerce')

# Add a column to show total gross income
gross_data['total_gross'] = gross_data['domestic_gross'] + gross_data['foreign_gross']

# Convert the values in the 'total_gross' column to int64
gross_data['total_gross'] = gross_data['total_gross'].astype('int64')

# Display the 'total_gross' column
print(gross_data['total_gross'])


0       1067000000
1       1025500000
2        960300000
3        828300000
4        752600000
           ...    
3382          6200
3383          4800
3384          2500
3385          2400
3386          1700
Name: total_gross, Length: 3387, dtype: int64


In [59]:
# Determine how many movies each studio has produced
prd_movies =gross_data['studio'].value_counts()
# convert the ouput to a dataframe
df_prd_movies = gross_data['studio'].value_counts()
df_prd_movies = pd.DataFrame(prd_movies)
df_prd_movies = df_prd_movies.reset_index()
df_prd_movies.columns = ['studio_name', 'total_movies_produced']
df_prd_movies

,studio_name,total_movies_produced
0,IFC,166
1,Uni.,147
2,WB,140
3,Fox,136
4,Magn.,136
...,...,...
252,E1,1
253,PI,1
254,ELS,1
255,PalT,1


- The above code indicates that most of the movies are produced by the' IFC' studios followed by Uni
WB,Fox and Magn.
- Our interest is also to find out how this top perfoming studios are doing in terms of the gross they receive.

In [60]:
#group the data using the studios and their total gross
grouped=gross_data.groupby('studio').sum('total_gross')
grouped


,domestic_gross,foreign_gross,year,total_gross
studio,,,,
3D,6100000.0,9900000.0,2010,16000000
A23,164200.0,0.0,4024,164200
A24,324194200.0,238462200.0,98754,562656400
ADC,248200.0,0.0,4032,248200
AF,2142900.0,3500000.0,12080,5642900
...,...,...,...,...
XL,458000.0,0.0,4027,458000
YFG,1100000.0,0.0,2016,1100000
Yash,31631400.0,272825100.0,28194,304456500


In [61]:
# Create a new dataframe by joining the gross_data dataframe with the df_prd_movies dataframe
joined=grouped.join(df_prd_movies.set_index("studio_name"))
joined=joined.drop('year', axis=1)
joined['avg_gross']=joined['total_gross']/joined['total_movies_produced']

# Change data type to integer
joined=joined.astype('int64')

# Sort the new dataframe by avg_gross
joined.sort_values('avg_gross', ascending=False)

,domestic_gross,foreign_gross,total_gross,total_movies_produced,avg_gross
studio,,,,,
HC,2700000,867600000,870300000,1,870300000
P/DW,1682900000,3393600000,5076500000,10,507650000
BV,18419029199,25793854700,44212883898,106,417102678
GrtIndia,20200000,234000000,254200000,1,254200000
WB (NL),3995699999,6339000000,10334699999,45,229659999
...,...,...,...,...,...
BSM,5100,0,5100,1,5100
TAFC,4600,0,4600,1,4600
Hiber,3600,0,3600,1,3600


The code above sorts the dataframe starting with the studio with the highest average gross to that with the lowest average gross.